# Alembic

`alembic` is a library that we can use in Python to manage our the versions of our PSQL database.

This allows us to upgrade the definitions of our database such as:
- Adding/Removing Columns
- Adding/Removing Tables
- Modifying some column properties (such as Unique, Nullable, Primary Key, Foreign Key, Data Type, etc.)
- Converting existing column value/property (such as VARCHAR to INT or vice versa)

To install use the following commands:
```sh
pip install alembic
```
- If there are issues during installation, consult me or Google because the issues vary for each environment.


More details can be found here:

https://alembic.sqlalchemy.org/en/latest/front.html#installation

## Alembic and SQLAlchemy

Alembic works with SQLAlchemy to autogenerate some of the Column and Table changes.

For example, if you add a column to an existing table from the SQLAlchemy models, alembic "autogenerate" can be used to create the script which can be ran to make the actual update to the database.

If necessary, it's possible to update the autogenerated script. For example, if in addition to a new column, we also need to populate it with some values, we will need to add some "insert" or "update" script alongside the generated one so that once the column is added, the expected values are also populated accordingly.

## Initializing Alembic
```sh
alembic init alembic
```

This creates the default directory called `alembic` and populates it with the default files. Notable files are:
- `alembic.ini`
    - This contains the configurations necessary for alembic to run
    - It also contains the default parameters to connect to the database such as the hostname, database name, database username and database password
- `alembic/env.py`
    - This is ran everytime we do an alembic migration
    - This is where we can override the default database credentials to avoid hardcoding them in the `alembic.ini`
- `alembic/versions/`
    - This will contain the python scripts to be ran by the "migration" to perform the database update

## Preparing the alembic connection

Since alembic will need to connect to the database, we will need to show it how to do so.

We can either update `alembic.ini` or update `alembic/env.py`. To avoid hardcoding values, we will update `alembic/env.py` instead so that we can programmatically fetch the credentials from the environment variables. This is shown in the snippet below:


```py
...

load_dotenv()  # take environment variables from .env.

database_host = environ.get('DB_HOST')
database_name = environ.get('DB_NAME')
database_username = environ.get('DB_USERNAME')
database_password = environ.get('DB_PASSWORD')

# this is the Alembic Config object, which provides
# access to the values within the .ini file in use.
config = context.config

# Interpret the config file for Python logging.
# This line sets up loggers basically.
if config.config_file_name is not None:
    fileConfig(config.config_file_name)

config.set_section_option(
    config.config_ini_section,
    "sqlalchemy.url",
    f'postgresql://{database_username}:{database_password}'
    f'@{database_host}:5432/{database_name}'
)

...
```

## Showing alembic the models to use

Now that alembic can connect to the database, we also need to show alembic the models that it needs to watch out for when autogenerating the migration script. This is useful for us so that we don't need to manually write the SQL statements and allow us to make sure that there's minimal differences between our models and what's in the database.

In the `alembic/env.py`, there is a variable called `target_metadata`. This should contain the object that can list out all of the model definitions that we have made. Luckily, SQLAlchemy allows us to automatically list them as all and is called "metadata". We can now update `alembic/env.py` as shown in the snippet below:
```py
...

from patents_view_api.models import Base

...

target_metadata = Base.metadata

...
```

## Creating a revision

Now that we have setup the connection, we can now easily create revisions by running the following command:
```sh
alembic revision --autogenerate
```

You can also add comments to the revision for easier tracking using the `-m` command:
```sh
alembic revision --autogenerate -m 'your comment here'
```

After running the command above, `alembic/versions/` directory will contain the newly generated script.

However, it is not performed automatically, we need another command to do that.

## The revision script

Below is a sample revision script that is autogenerated by alembic:

```py
"""another_test

Revision ID: 7b9ab09300c2
Revises: 0b707a7875f7
Create Date: 2022-04-21 20:00:55.721348

"""
from alembic import op
import sqlalchemy as sa


# revision identifiers, used by Alembic.
revision = '7b9ab09300c2'
down_revision = '0b707a7875f7'
branch_labels = None
depends_on = None


def upgrade():
    # ### commands auto generated by Alembic - please adjust! ###
    op.add_column('inventors', sa.Column('new_column', sa.VARCHAR(), nullable=True))
    # ### end Alembic commands ###


def downgrade():
    # ### commands auto generated by Alembic - please adjust! ###
    op.drop_column('inventors', 'new_column')
    # ### end Alembic commands ###

```

Some takeaways here are:
- `revision` contains the version "hash" for this script (you can think of this as the "version number")
- `down_revision` defines the previous version's hash
- `branch_label` allows you to define what "branch" from the revision is this script for (similar to Git branches)
    - One usage of this would be if you have different production and test databases
        - If you have a single database, you may need to create "test" revisions that are not production ready yet, so you would need to make another branch to do that
        - Similarly, another engineer would do the same and you won't impact each other since you're on different branches
        - Whenever either one of them finishes their "branched revisions", they will just need to point it back to "production", without affected the tests that the other employee is doing
- `depends_on` allows different branches to "create dependencies"
    - For example, you need to use revisions that are from another team, but their revision is not in production yet, you can just create a dependency to their branch to allow you to test your own changes.
    - Once both of your branches are ready, you can then merge back to "production"
- `upgrade()` contains the actual code that alembic will execute when upgrading to this revision
- `downgrade()` contains the actual code that alembic will execute when downgrading to the previous revision (usually the exact opposite of `upgrade()`)

## Running the "upgrade" migration script

Since the migration script changes database configurations, alembic allows us to run it directly to the database OR run it manually as an SQL command.


### Online Upgrade Migration
The first one is called "online" migration. This directly commits the changes to the SQL server.

We do this by running the following command:
```sh
alembic upgrade 7b9ab09300c2
```

This assumes that you know the revision hash beforehand, which is usually necessary to make sure that you explicitly know the version you want to use. However, alembic allows implicit "next" version migration by using the following command:
```sh
alembic upgrade +1
```

Doing so will tell alembic that it should migrate to the next version, without you explicitly stating what the next version is.

### Offline Upgrade Migration
The other one is called "offline" migration. This only creates the actual SQL command that you will need to run manually on the SQL server.

We do this by adding `--sql` on the command like so:
```sh
alembic upgrade 0b707a7875f7:7b9ab09300c2 --sql
```

Because this is offline, it's expected that you don't connect to the database, so, alembic doesn't actually know the current revision in the database.

This means you need to tell the starting revision (`0b707a7875f7` in this case) and the target "up" revision (`7b9ab09300c2` in this case)
```

## Running the "downgrade" migration script

Since the migration script changes database configurations, alembic allows us to run it directly to the database OR run it manually as an SQL command.


### Online Downgrade Migration
The first one is called "online" migration. This directly commits the changes to the SQL server.

We do this by running the following command:
```sh
alembic downgrade 0b707a7875f7
```

This assumes that you know the revision hash beforehand, which is usually necessary to make sure that you explicitly know the version you want to use. However, alembic allows implicit "previous" version migration by using the following command:
```sh
alembic downgrade -1
```

Doing so will tell alembic that it should migrate to the previous version, without you explicitly stating what the previous version is.

### Offline Downgrade Migration
The other one is called "offline" migration. This only creates the actual SQL command that you will need to run manually on the SQL server.

We do this by adding `--sql` on the command like so:
```sh
alembic downgrade 7b9ab09300c2:0b707a7875f7 --sql
```

Because this is offline, it's expected that you don't connect to the database, so, alembic doesn't actually know the current revision in the database.

This means you need to tell the starting revision (`7b9ab09300c2` in this case) and the target "down" revision (`0b707a7875f7` in this case)